# **Template OP on salish**

In [11]:
%matplotlib inline

import sys
import xarray as xr
import numpy as np
import os
import math
from datetime import datetime, timedelta
from parcels import FieldSet, Field, VectorField, ParticleSet, JITParticle, ErrorCode, AdvectionRK4_3D

sys.path.append('/home/jvalenti/MOAD/analysis-jose/notebooks/parcels')
from Kernels import DeleteParticle, Stokes_drift
from OP_functions import *

In [12]:
# Define paths
local = 0 #Set to 0 when working on server
paths = path(local)

## Definitions

In [13]:
start = datetime(2019, 12, 10) #Start date
# Set Time length [days] and timestep [seconds]
length = 10
duration = timedelta(days=length)
dt = 90 #toggle between - or + to pick backwards or forwards
N = 6 # number of deploying locations
n = 100 # 1000   # number of particles per location
dmin = [0,0,0,0,0,70] #minimum depth
dd = 20 #max depth difference from dmin
x_offset, y_offset, zvals = p_deploy(N,n,dmin,dd)

In [14]:
from parcels import Variable

class PEParticle(JITParticle):         # Define a new particle class
    ro = Variable('ro', initial = 960)             # desnsity PE
    size = Variable('size', initial = 5e-5)

class BParticle(JITParticle):         # Define a new particle class
    beached = Variable('beached', initial = 0) #Keep track of beached particles

In [15]:
Dat=xr.open_dataset(paths['coords'],decode_times=False)
outf_lat=Dat['nav_lat'][445,304]
outf_lon=Dat['nav_lon'][445,304]

In [16]:
lon = np.zeros([N,n])
lat = np.zeros([N,n])
# Execute run
clon, clat = [-123.901172,-125.155849,-123.207648,-122.427508,-123.399769,float(outf_lon)], [49.186308,49.975326,49.305448,47.622403,48.399420,float(outf_lat)]  # choose horizontal centre of the particle cloud
for i in range(N):
    lon[i,:]=(clon[i] + x_offset[i,:])
    lat[i,:]=(clat[i] + y_offset[i,:])
z = zvals

In [17]:
#Set start date time and the name of the output file
name = 'Wavesss' #name output file
daterange = [start+timedelta(days=i) for i in range(length)]
fn =  name + '_'.join(d.strftime('%Y%m%d')+'_1n' for d in [start, start+duration]) + '.nc'
outfile = os.path.join(paths['out'], fn)
print(outfile)

/home/jvalenti/MOAD/analysis-jose/notebooks/results/Wavesss20191210_1n_20191220_1n.nc


## Simulation

In [18]:
#Fill in the list of variables that you want to use as fields
varlist=['U','V','W','R']
filenames,variables,dimensions=filename_set(start,length,varlist,local)
field_set=FieldSet.from_nemo(filenames, variables, dimensions, allow_time_extrapolation=True)

varlist=['US','VS','WL']
filenames,variables,dimensions=filename_set(start,length,varlist,local)

#dimensions = {'lon': coord['longitude'], 'lat': coord['latitude'], 'time': 'time'}
us = Field.from_netcdf(filenames['US'], variables['US'], dimensions,allow_time_extrapolation=True)
vs = Field.from_netcdf(filenames['VS'], variables['VS'], dimensions,allow_time_extrapolation=True)
wl = Field.from_netcdf(filenames['WL'], variables['WL'], dimensions,allow_time_extrapolation=True)
field_set.add_field(us)
field_set.add_field(vs)
field_set.add_field(wl)
field_set.add_vector_field(VectorField("stokes", us, vs, wl))

In [19]:
# #Load Salish output as fields
#field_set = FieldSet.from_nemo(filenames, variables, dimensions, allow_time_extrapolation=True)
pset = ParticleSet.from_list(field_set, JITParticle, lon=lon, lat=lat, depth=z, time=start+timedelta(hours=2))

p_new = pset.Kernel(Stokes_drift) #uncomment if you want to use additional kernels, also change the value of pset.Kernel to "AdvectionRK4_3D + p_flt"
##pset.computeTimeChunk(allow_time_extrapolation=1)
pset.execute(AdvectionRK4_3D + p_new, 
             runtime=duration, 
             dt=dt,
             output_file=pset.ParticleFile(name=outfile, outputdt=timedelta(hours=1)),
             recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle})

INFO: Compiled ArrayJITParticleAdvectionRK4_3DStokes_drift ==> /tmp/parcels-2894/libeee3cfc741cb5dafdc90fa9f7ec8d6eb_0.so


KeyboardInterrupt: 